<a href="https://colab.research.google.com/github/thargro5/Machine-Learning/blob/main/MLHW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import time

# Load and preprocess the dataset (same as before)
df = pd.read_csv("https://github.com/HamedTabkhi/Intro-to-ML/raw/main/Dataset/Housing.csv")

numerical_features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price']
categorical_features = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

X = df[numerical_features + categorical_features]
y = df['price']

# Preprocess
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Ensure X is converted to a dense array after transformation
X = preprocessor.fit_transform(X)

# If it's a sparse matrix (depends on your pipeline configuration), convert it:
if hasattr(X, "toarray"):  # Only call .toarray() if it's sparse
    X = X.toarray()


y = y.values.reshape(-1, 1)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert into PyTorch tensors (directly from dense NumPy arrays)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# Define the neural network
class HousingNN(nn.Module):
    def __init__(self, input_dim):
        super(HousingNN, self).__init__()
        self.hidden = nn.Linear(input_dim, 8)  # Hidden layer with 8 nodes
        self.output = nn.Linear(8, 1)         # Output layer (regression)
        self.relu = nn.ReLU()                # Activation function for hidden layer

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.output(x)
        return x

# Initialize the model
input_dim = X_train_tensor.shape[1]
model = HousingNN(input_dim)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 2000
losses = []
val_accuracies = []

start_time = time.time()

for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    losses.append(loss.item())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation every 500 epochs
    if (epoch + 1) % 500 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)
            val_accuracy = 1 - (val_loss / torch.var(y_val_tensor))
            val_accuracies.append(val_accuracy.item())
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

training_time = time.time() - start_time

# Print training time
print(f"Training time: {training_time:.2f} seconds")

Epoch [500/2000], Loss: 25202301861888.0000, Val Accuracy: -4.8987
Epoch [1000/2000], Loss: 25106824822784.0000, Val Accuracy: -4.8766
Epoch [1500/2000], Loss: 24959143378944.0000, Val Accuracy: -4.8425
Epoch [2000/2000], Loss: 24767216222208.0000, Val Accuracy: -4.7980
Training time: 4.01 seconds


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.model_selection import train_test_split

# Define the extended neural network
class ExtendedHousingNN(nn.Module):
    def __init__(self, input_dim):
        super(ExtendedHousingNN, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 8)  # Hidden layer 1
        self.hidden2 = nn.Linear(8, 8)         # Hidden layer 2
        self.hidden3 = nn.Linear(8, 4)         # Hidden layer 3
        self.output = nn.Linear(4, 1)          # Output layer (regression)
        self.relu = nn.ReLU()                 # Activation function

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        x = self.output(x)
        return x

# Initialize the model
input_dim = X_train_tensor.shape[1]  # Number of features
model = ExtendedHousingNN(input_dim)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 3000  # Increased epochs to avoid underfitting
losses = []
val_accuracies = []

start_time = time.time()

for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    losses.append(loss.item())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation every 500 epochs
    if (epoch + 1) % 500 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)
            val_accuracy = 1 - (val_loss / torch.var(y_val_tensor))
            val_accuracies.append(val_accuracy.item())
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

training_time = time.time() - start_time

# Print training time
print(f"Training time: {training_time:.2f} seconds")

Epoch [500/3000], Loss: 25234744803328.0000, Val Accuracy: -4.9062
Epoch [1000/3000], Loss: 25234698665984.0000, Val Accuracy: -4.9062
Epoch [1500/3000], Loss: 25234648334336.0000, Val Accuracy: -4.9062
Epoch [2000/3000], Loss: 25234602196992.0000, Val Accuracy: -4.9062
Epoch [2500/3000], Loss: 25234556059648.0000, Val Accuracy: -4.9062
Epoch [3000/3000], Loss: 25234509922304.0000, Val Accuracy: -4.9062
Training time: 4.91 seconds


In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import time

# Load the dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)  # Add dimension for binary classification
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

class CancerNN(nn.Module):
    def __init__(self, input_dim):
        super(CancerNN, self).__init__()
        self.hidden = nn.Linear(input_dim, 32)  # Hidden layer with 32 nodes
        self.output = nn.Linear(32, 1)          # Output layer (binary classification)
        self.relu = nn.ReLU()                  # Activation function
        self.sigmoid = nn.Sigmoid()            # Output activation (binary probs)

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

# Initialize the model
input_dim = X_train_tensor.shape[1]  # Number of features
model = CancerNN(input_dim)

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 1000
losses = []
val_accuracies = []

start_time = time.time()

for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    losses.append(loss.item())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation accuracy every 100 epochs
    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_predictions = (val_outputs > 0.5).float()
            val_accuracy = (val_predictions == y_val_tensor).float().mean()
            val_accuracies.append(val_accuracy.item())
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

training_time = time.time() - start_time

# Print training time
print(f"Training time: {training_time:.2f} seconds")

Epoch [100/1000], Loss: 0.1706, Val Accuracy: 0.9737
Epoch [200/1000], Loss: 0.0873, Val Accuracy: 0.9825
Epoch [300/1000], Loss: 0.0630, Val Accuracy: 0.9912
Epoch [400/1000], Loss: 0.0502, Val Accuracy: 0.9825
Epoch [500/1000], Loss: 0.0411, Val Accuracy: 0.9825
Epoch [600/1000], Loss: 0.0338, Val Accuracy: 0.9825
Epoch [700/1000], Loss: 0.0275, Val Accuracy: 0.9825
Epoch [800/1000], Loss: 0.0222, Val Accuracy: 0.9825
Epoch [900/1000], Loss: 0.0179, Val Accuracy: 0.9737
Epoch [1000/1000], Loss: 0.0146, Val Accuracy: 0.9649
Training time: 1.49 seconds


In [ ]:
class ExtendedCancerNN(nn.Module):
    def __init__(self, input_dim):
        super(ExtendedCancerNN, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 32)  # Hidden layer 1
        self.hidden2 = nn.Linear(32, 16)         # Hidden layer 2
        self.hidden3 = nn.Linear(16, 8)         # Hidden layer 3
        self.output = nn.Linear(8, 1)           # Output layer
        self.relu = nn.ReLU()                   # Activation function
        self.sigmoid = nn.Sigmoid()             # Sigmoid for binary classification

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        x = self.sigmoid(self.output(x))
        return x

# Initialize the model
input_dim = X_train_tensor.shape[1]  # Number of features
extended_model = ExtendedCancerNN(input_dim)

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(extended_model.parameters(), lr=0.001)

epochs = 1000
losses = []
val_accuracies = []

start_time = time.time()

for epoch in range(epochs):
    extended_model.train()

    # Forward pass
    outputs = extended_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    losses.append(loss.item())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation accuracy every 100 epochs
    if (epoch + 1) % 100 == 0:
        extended_model.eval()
        with torch.no_grad():
            val_outputs = extended_model(X_val_tensor)
            val_predictions = (val_outputs > 0.5).float()
            val_accuracy = (val_predictions == y_val_tensor).float().mean()
            val_accuracies.append(val_accuracy.item())
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

training_time = time.time() - start_time

# Print training time
print(f"Training time: {training_time:.2f} seconds")



Epoch [100/1000], Loss: 0.1991, Val Accuracy: 0.9912
Epoch [200/1000], Loss: 0.0425, Val Accuracy: 0.9825
Epoch [300/1000], Loss: 0.0163, Val Accuracy: 0.9825
Epoch [400/1000], Loss: 0.0068, Val Accuracy: 0.9825
Epoch [500/1000], Loss: 0.0031, Val Accuracy: 0.9825
Epoch [600/1000], Loss: 0.0016, Val Accuracy: 0.9825
Epoch [700/1000], Loss: 0.0008, Val Accuracy: 0.9825
Epoch [800/1000], Loss: 0.0005, Val Accuracy: 0.9825
Epoch [900/1000], Loss: 0.0003, Val Accuracy: 0.9825
Epoch [1000/1000], Loss: 0.0002, Val Accuracy: 0.9737
Training time: 6.71 seconds


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Data transformation and normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing to [-1, 1]
])

# Load training and testing data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

class CIFAR10NN(nn.Module):
    def __init__(self):
        super(CIFAR10NN, self).__init__()
        self.flatten = nn.Flatten()
        self.hidden = nn.Linear(32*32*3, 256)  # Input size: 32x32x3 (CIFAR-10 images)
        self.relu = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 10 classes for CIFAR-10
        self.softmax = nn.Softmax(dim=1)  # Softmax to get probabilities

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.hidden(x))
        x = self.softmax(self.output(x))
        return x

# Initialize the model
model = CIFAR10NN()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
train_loss_history = []
train_accuracy_history = []

start_time = time.time()  # Start timer for training

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()

        # Update weights
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(trainloader)
    accuracy = 100 * correct / total
    train_loss_history.append(avg_loss)
    train_accuracy_history.append(accuracy)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Record the training time
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")

# Evaluate the model on the test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

100%|██████████| 170M/170M [00:01<00:00, 98.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [10/100], Loss: 1.9436, Accuracy: 51.54%
Epoch [20/100], Loss: 1.9128, Accuracy: 54.65%
Epoch [30/100], Loss: 1.8804, Accuracy: 57.96%
Epoch [40/100], Loss: 1.8624, Accuracy: 59.74%
Epoch [50/100], Loss: 1.8576, Accuracy: 60.24%
Epoch [60/100], Loss: 1.8509, Accuracy: 60.91%
Epoch [70/100], Loss: 1.8394, Accuracy: 62.10%
Epoch [80/100], Loss: 1.8338, Accuracy: 62.68%
Epoch [90/100], Loss: 1.8335, Accuracy: 62.69%
Epoch [100/100], Loss: 1.8263, Accuracy: 63.44%
Training time: 3344.82 seconds
Test Accuracy: 50.80%
